# Programação Matemática (GCC118)
## Universidade Federal de Lavras (UFLA)
### Instituto de Ciências Exatas e Tecnológicas

#### Grupo
- Ranulfo Mascari Neto
- Heitor Rodrigues Sabino

## Problema 2: Empresa Transportadora de Óleos

Uma empresa transporta óleo de uma estação $A$ para uma estação $B$. Existe uma rota **norte** e uma rota **sul**, ambas com uma estação intermediária. Ainda existe uma ligação entre a estação intermediária sul e a estação intermediária norte. Abaixo estão as capacidades máximas de transporte (em barris por dia) para cada trecho do percurso:

- **De A para Norte**: 300 barris/dia.
- **De A para Sul**: 500 barris/dia.
- **De Sul para Norte**: 300 barris/dia.
- **De Norte para B**: 400 barris/dia.
- **De Sul para B**: 300 barris/dia.

Formule um programa linear, utilizando *gurobipy*, que **maximiza os barris por dia transportados de $A$ para $B$**. Faça uma **análise de sensibilidade**, e discuta o que tal análise reportará.

### Modelagem Matemática
A seguir, apresentaremos a modelagem matemática deste problema, especificando os principais elementos da modelagem de um problema de programação matemática: $(i)$ parâmetros (dados); $(ii)$ variáveis de decisão; $(iii)$ modelagem, composta por uma função objetivo, restrições do problema e restrições de domínio das variáveis de decisão.

---

*Observação: A maximização do transporte de barris de óleo de $A$ para $B$ pode ser modelada como um problema de fluxo máximo em um grafo direcionado. Cada nó representa uma estação, e cada aresta representa a capacidade máxima de transporte de barris de óleo por dia.*

---

Segue o problema de otimização linear modelado como um problema de fluxo máximo:

#### $(i)$ Parâmetros

- **Capacidade de Transporte**:
  - $c_{ij} \in \mathbb{R}_+$: capacidade máxima de transporte de barris por dia entre os nós $i$ e $j$, onde $i, j \in \{A, N, S, B\}$.
    - $c_{A,N} = 300$ (de A para Norte)
    - $c_{A,S} = 500$ (de A para Sul)
    - $c_{S,N} = 300$ (de Sul para Norte)
    - $c_{N,B} = 400$ (de Norte para B)
    - $c_{S,B} = 300$ (de Sul para B)

- **Nós**:
  - **Estações**:
    - $\{A, N, S, B\}$: conjunto dos nós do grafo, representando as estações A, Norte, Sul e B.

---

#### $(ii)$ Variáveis de Decisão

- $x_{ij} \in \mathbb{R}_+$: fluxo de barris de óleo transportados por dia entre os nós $i$ e $j$, onde $i, j \in \{A, N, S, B\}$.
  - $x_{A,N}$: fluxo de A para Norte.
  - $x_{A,S}$: fluxo de A para Sul.
  - $x_{S,N}$: fluxo de Sul para Norte.
  - $x_{N,B}$: fluxo de Norte para B.
  - $x_{S,B}$: fluxo de Sul para B.

---

#### $(iii)$ Modelagem

- **Função Objetivo**:
  - **Maximizar o Fluxo Total**:
  $$
  \max x_{N,B} + x_{S,B}
  $$
    - Maximiza o fluxo total de barris transportados de A para B.

- **Restrições**:
  1. **Capacidade Máxima nas Arestas**:
     - Para cada ligação direta entre os nós:
     $$
     x_{A,N} \leq 300, \quad x_{A,S} \leq 500, \quad x_{S,N} \leq 300, \quad x_{N,B} \leq 400, \quad x_{S,B} \leq 300
     $$
     - Garante que o fluxo em cada aresta não ultrapasse sua capacidade máxima.

  2. **Conservação de Fluxo nos Nós**:
     - **Norte**:
     $$
     x_{A,N} + x_{S,N} = x_{N,B}
     $$
     - **Sul**:
     $$
     x_{A,S} = x_{S,N} + x_{S,B}
     $$

  3. **Não Negatividade**:
     $$
     x_{ij} \geq 0, \quad \forall i, j \in \{A, N, S, B\}
     $$
     - O fluxo transportado em cada ligação deve ser não negativo.

---

*Observação: Na modelagem deste problema, a **função objetivo** foi definida como a soma dos **fluxos de entrada em $B$** ($x_{NB} + x_{SB}$) e não como a soma de todos os fluxos no grafo. Essa escolha reflete o objetivo principal do problema: **maximizar a quantidade de barris de óleo transportados da origem ($A$) ao destino final ($B$)**.*

---

### Modelo Matemático do Problema

#### **Função Objetivo**
**Maximizar o Fluxo Total**:
$$
\max x_{N,B} + x_{S,B}
$$

#### **Restrições**
1. **Capacidade das Arestas**:
   $$
   x_{A,N} \leq 300, \quad x_{A,S} \leq 500, \quad x_{S,N} \leq 300, \quad x_{N,B} \leq 400, \quad x_{S,B} \leq 300
   $$

2. **Conservação de Fluxo no Nó Norte**:
   $$
   x_{A,N} + x_{S,N} = x_{N,B}
   $$

3. **Conservação de Fluxo no Nó Sul**:
   $$
   x_{A,S} = x_{S,N} + x_{S,B}
   $$

4. **Não Negatividade**:
   $$
   x_{ij} \geq 0, \quad \forall i, j \in \{A, N, S, B\}
   $$

---

### Resolução do Problema

#### Instalação da biblioteca Gurobi

In [1]:
!pip install gurobipy

In [2]:
from gurobipy import Model, GRB

# Criando o modelo
model = Model("Empresa Transportadora de Óleos")

# Variáveis de decisão
x_AN = model.addVar(lb=0, name="x_AN")  # Fluxo de A para Norte
x_AS = model.addVar(lb=0, name="x_AS")  # Fluxo de A para Sul
x_SN = model.addVar(lb=0, name="x_SN")  # Fluxo de Sul para Norte
x_NB = model.addVar(lb=0, name="x_NB")  # Fluxo de Norte para B
x_SB = model.addVar(lb=0, name="x_SB")  # Fluxo de Sul para B

# Função objetivo: Maximizar o fluxo para o nó B
model.setObjective(x_NB + x_SB, GRB.MAXIMIZE)

# Restrições de capacidade para cada aresta
model.addConstr(x_AN <= 300, name="c_AN")  # Capacidade de A para Norte
model.addConstr(x_AS <= 500, name="c_AS")  # Capacidade de A para Sul
model.addConstr(x_SN <= 300, name="c_SN")  # Capacidade de Sul para Norte
model.addConstr(x_NB <= 400, name="c_NB")  # Capacidade de Norte para B
model.addConstr(x_SB <= 300, name="c_SB")  # Capacidade de Sul para B

# Restrições de conservação de fluxo
model.addConstr(x_AN + x_SN == x_NB, name="Fluxo_Norte")  # Fluxo no nó Norte
model.addConstr(x_AS == x_SN + x_SB, name="Fluxo_Sul")    # Fluxo no nó Sul

# Otimização
model.optimize()

# Resultados
if model.status == GRB.OPTIMAL:
    print(f"Fluxo Máximo: {model.objVal:.2f}")
    for x in model.getVars():
        print(f"{x.varName}: {x.X}")


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 5 columns and 11 nonzeros
Model fingerprint: 0xdfb8879e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 5e+02]
Presolve removed 5 rows and 1 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+02   1.250000e+01   0.000000e+00      0s
       1    7.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.000000000e+02
Fluxo Máximo: 700.00
x_AN: 300.0
x_AS: 400.0
x_SN: 100.0
x_NB:

### Análise de Sensibilidade

#### Implementação da Análise de Sensibilidade com a Biblioteca *GurobiPy*

In [3]:
# Análise de Sensibilidade
if model.status == GRB.OPTIMAL:

  # Valores das variáveis duais (Preços Sombra)
  print("\nSolução Ótima Dual:")
  for c in model.getConstrs():
      print(f"\t{c.ConstrName} - Valor Dual (Pi): {c.Pi}")

  # Intervalos de Viabilidade dos Recursos (Capacidades das Arestas e Conservação de Fluxo)
  print("\nIntervalos de Recursos (Capacidades das Arestas e Conservação de Fluxo):")
  for c in model.getConstrs():
      print(f"\t{c.ConstrName} [SARHSLower: {c.SARHSLow}, SARHSUpper: {c.SARHSUp}]")

  # Intervalos de Viabilidade dos Custos (Objetivo)
  print("\nIntervalos de Custos (Vetor de Custos):")
  for v in model.getVars():
      print(f"\t{v.varName} [SAObjLow: {v.SAObjLow}, SAObjUp: {v.SAObjUp}]")


Solução Ótima Dual:
	c_AN - Valor Dual (Pi): -0.0
	c_AS - Valor Dual (Pi): 0.0
	c_SN - Valor Dual (Pi): 0.0
	c_NB - Valor Dual (Pi): 1.0
	c_SB - Valor Dual (Pi): 1.0
	Fluxo_Norte - Valor Dual (Pi): -0.0
	Fluxo_Sul - Valor Dual (Pi): -0.0

Intervalos de Recursos (Capacidades das Arestas e Conservação de Fluxo):
	c_AN [SARHSLower: 200.0, SARHSUpper: 400.0]
	c_AS [SARHSLower: 400.0, SARHSUpper: inf]
	c_SN [SARHSLower: 100.0, SARHSUpper: inf]
	c_NB [SARHSLower: 300.0, SARHSUpper: 500.0]
	c_SB [SARHSLower: 0.0, SARHSUpper: 400.0]
	Fluxo_Norte [SARHSLower: -100.0, SARHSUpper: 100.0]
	Fluxo_Sul [SARHSLower: -400.0, SARHSUpper: 100.0]

Intervalos de Custos (Vetor de Custos):
	x_AN [SAObjLow: -0.0, SAObjUp: inf]
	x_AS [SAObjLow: -1.0, SAObjUp: 0.0]
	x_SN [SAObjLow: -1.0, SAObjUp: 0.0]
	x_NB [SAObjLow: -0.0, SAObjUp: inf]
	x_SB [SAObjLow: -0.0, SAObjUp: inf]


#### Interpretação da Análise de Sensibilidade

1. **Arestas Críticas**:
   - As arestas **$c_{NB}$** (Norte -> B) e **$c_{SB}$** (Sul -> B) são gargalos no sistema de transporte. Isso significa que suas capacidades estão totalmente utilizadas e limitam o fluxo máximo.
   - Investir em aumentar a capacidade dessas arestas pode resultar diretamente em um aumento do fluxo máximo transportado.

2. **Arestas com Folga**:
   - As arestas **$c_{AN}$** (A -> Norte), **$c_{AS}$** (A -> Sul) e **$c_{SN}$** (Sul -> Norte) possuem folga em suas capacidades ou não estão totalmente utilizadas. Isso indica que aumentar suas capacidades não afetará o fluxo máximo atual.

3. **Análise de Gargalos**:
   - A análise de sensibilidade confirma que o limite do fluxo máximo é imposto pelas arestas de saída para o destino final (**Norte -> B** e **Sul -> B**).
   - A capacidade da aresta intermediária (**Sul -> Norte**) também não é um gargalo, e há margem para aumentar o fluxo entre os intermediários sem afetar a solução ótima.

4. **Investimentos Futuros**:
   - Um aumento de 1 unidade na capacidade das arestas **$c_{NB}$** ou **$c_{SB}$** aumentaria diretamente o fluxo total em 1 barril por dia.
   - Não há necessidade de expandir as capacidades de **$c_{AN}$**, **$c_{AS}$** ou **$c_{SN}$** no momento, pois essas arestas não são limitantes.
  